In [18]:
!pip install tweepy

  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)


In [ ]:
!pip install spacy
!pip install gensim
!pip install pyldavis
!pip install wordcloud
!pip install textblob
!pip install textstat
!pip install nltk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nltk, time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
import collections, itertools

In [20]:
import os
import tweepy as tw
import pandas as pd

In [24]:
consumer_key= 'QxA50wplG6jBYLpNMBDJCYjcH'
consumer_secret= 'Js4FSedtS30D6HEItelpIc3mNtu2NgFKsO04rbRjj73WykIlGn'
access_token= '1329496543779573760-tOMLouKs116LWwgXZF7oZdtGU2I8an'
access_token_secret= 'c2cPhglkr87O15nLPkxZlNVMPNDwFqf6oZRDiD8hUpPR6'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [98]:
search_words = "koronawirus"
date_since = "2020-01-01"

tweets = tw.Cursor(api.search,
              q=search_words,
              tweet_mode='extended',
              lang="pl",
              since=date_since).items(10000)

In [99]:
list_tweet = []
for tweet in tweets:
    list_tweet.append(tweet.full_text)

In [101]:
len(list_tweet)

10000

In [104]:
df.head()

,0
0,2/2\nKoronawirus wciąż wstrzymuje ruch w halac...
1,RT @sondypolityczne: #sonda | czy protesty #St...
2,1/2\nKoronawirus wciąż wstrzymuje ruch w halac...
3,RT @agnieszkawolsk9: Kto i kiedy wybrał Polskę...
4,RT @agnieszkawolsk9: Kto i kiedy wybrał Polskę...


In [105]:
df.to_csv("tweets.csv", index=False)

In [102]:
df = pd.DataFrame(list_tweet)

In [103]:
import spacy
nlp = spacy.load('pl_core_news_sm') 

In [72]:
doc = nlp("Granice mojego języka oznaczają granice mojego świata")
for token in doc:
    print(token.text, token.lemma_, token.tag_)

Granice granica SUBST
mojego mój ADJ
języka język SUBST
oznaczają oznaczać FIN
granice granica SUBST
mojego mój ADJ
świata świat SUBST


In [70]:
from spacy import displacy
displacy.render(doc, style="dep")

In [74]:
list_tweet[6]

'Blokowanie unijnego budżetu i funduszu odbudowy po #koronawirus to nie tylko polityczne awanturnictwo. To odbierani… https://t.co/jovTkHbTfF'

In [ ]:
doc = nlp(list_tweet[1])


In [ ]:
def custom_tokenize(text):
    """Function that tokenizes text"""
    from nltk.tokenize import word_tokenize
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
In [4]:
def clean_up(data):
    """Function that cleans up the data into a shape that can be further used for modeling"""
    english = data[data['lang']=='en'] # extract only tweets in english language
    english.drop_duplicates() # drop duplicate tweets
    english['text'].dropna(inplace=True) # drop any rows with missing tweets
    tokenized = english['text'].apply(custom_tokenize) # Tokenize tweets
    lower_tokens = tokenized.apply(lambda x: [t.lower() for t in x]) # Convert tokens into lower case
    alpha_only = lower_tokens.apply(lambda x: [t for t in x if t.isalpha()]) # Remove punctuations
    no_stops = alpha_only.apply(lambda x: [t for t in x if t not in stopwords.words('english')]) # remove stop words
    no_stops.apply(lambda x: [x.remove(t) for t in x if t=='rt']) # remove acronym "rt"
    no_stops.apply(lambda x: [x.remove(t) for t in x if t=='https']) # remove acronym "https"
    no_stops.apply(lambda x: [x.remove(t) for t in x if t=='twitter']) # remove the word "twitter"
    no_stops.apply(lambda x: [x.remove(t) for t in x if t=='retweet']) # remove the word "retweet"
    return no_stops
In [5]:
start = time.time()
warnings.filterwarnings("ignore")
use_cols = ['text', 'lang']
path = 'C:\\Users\\Bauyrjan.Jyenis\\Capstone3\\all_tweets.csv'
data_iterator = pd.read_csv(path, usecols=use_cols, chunksize=50000)
chunk_list = []
for data_chunk in data_iterator:
    filtered_chunk = clean_up(data_chunk)
    chunk_list.append(filtered_chunk)
tidy_data = pd.concat(chunk_list)
end = time.time()
total=end-start
total #46min

In [ ]:
_ = tweets.str.len().hist(grid=False, figsize=(9,7), bins=20)
_ = plt.xlabel('Number of words')
_ = plt.ylabel('Frequency')
_ = plt.title('Number of words in each tweet')
plt.show()

In [ ]:
from gensim.corpora.dictionary import Dictionary
# Create a Dictionary from the tweets
start = time.time()

dictionary = Dictionary(tweets)

end = time.time()
total = end-start
total
start = time.time()
corpus = tweets.apply(lambda x: dictionary.doc2bow(x))

end = time.time()
total = end-start
total

In [ ]:
import collections, itertools
start = time.time()

total_word_count = collections.defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id]+=word_count
    
end = time.time()
total = end-start
total

In [ ]:
start = time.time()

sorted_bow = sorted(total_word_count.items(), key=lambda kv :kv[1], reverse=True)
words = []
word_counts = []
for word_id, word_count in sorted_bow[:20]:
    words.append(dictionary.get(word_id)) 
    word_counts.append(word_count)
    
end = time.time()
total = end-start
total

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sns.barplot(x=word_counts, y=words)
fig = plt.xlabel("Count")
fig = plt.ylabel('Words')
fig = plt.title('20 Most Frequent Words Across All Tweets')
plt.show()

In [ ]:
from nltk.util import ngrams
start = time.time()

bigrams = tweets.apply(lambda x: ngrams(x,2))

end = time.time()
total = end-start
total

bi_grams = bigrams.apply(lambda x: list(x))
top_20_bigrams = Counter(itertools.chain.from_iterable(bi_grams)).most_common(20)
top_bigram = pd.DataFrame(top_20_bigrams, columns=['bigram', 'frequency'])
top_bigram.set_index('bigram', inplace=True)
_ = plt.figure(figsize=(12,9))
_ = sns.barplot(x=top_bigram.frequency, y=top_bigram.index, orient='h')
_ = plt.xlabel('Frequency')
_ = plt.ylabel('Bigrams')
_ = plt.title('Top 20 Bigrams')
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS

stopwords = set(STOPWORDS)
def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)
    wordcloud=wordcloud.generate(str(data))
    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(tweets)